In [2]:
# Useful starting lines
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
from helpers import *
from surprise import *
import csv

%load_ext autoreload
%autoreload 2

# Load Data

In [3]:
from helpers import load_data, preprocess_data

path_dataset = "47b05e70-6076-44e8-96da-2530dc2187de_data_train.csv"
path_submission = "9b4d32bb-f99a-466f-95a1-0ab80048971c_sample_submission (2).csv"
ratings = load_data(path_dataset)
submission = load_submission(path_submission)
submission_row_col = submission[0]
submission_pos = submission[1]
print(ratings.shape)

number of items: 10000, number of users: 1000
(10000, 1000)


In [4]:
nb_nzi = len(ratings.nonzero()[0])
nb_nzu = len(ratings.nonzero()[1])

users = ratings.nonzero()[1]
items = ratings.nonzero()[0]

stars = []
for i in range(nb_nzi):
    stars.append(ratings[items[i],users[i]])

In [5]:
print(items[0],users[0],stars[0])

0 9 5.0


In [6]:
with open('surprise', 'w') as csvfile:
    fieldnames = ['item', 'user','ratings']
    writer = csv.DictWriter(csvfile, delimiter=";", fieldnames=fieldnames)
    for r1, r2,r3 in zip(items, users, stars):
        writer.writerow({'item':r1,'user':r2,'ratings':r3})

In [3]:
from surprise.model_selection import cross_validate
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# path to dataset file
file_path = 'surprise'

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='item user rating', sep=';')

data = Dataset.load_from_file(file_path, reader=reader)

In [4]:
from surprise import*
# Use the famous SVD algorithm.

sim_options = {'name' : 'pearson_baseline','shrinkage':1000}
sim_options_baseline = {'name' : 'pearson_baseline','shrinkage':100} 

algo1 = SVD(lr_all = 0.002, reg_pu = 0.05, reg_qi = 0.7)
algo2 = SVDpp(lr_all = 0.002, reg_pu = 0.1, reg_qi = 0.7)
algo3 = KNNBasic()
algo4 = KNNWithMeans(sim_options = sim_options, k = 1000)
algo5 = KNNWithZScore()
algo6 = KNNBaseline(k = 200, sim_options = sim_options_baseline)
algo7 = CoClustering()
algo8 = SlopeOne()
algo9 = NMF()
# Run 5-fold cross-validation and print results.
#cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
#cross_validate(BaselineOnly(), data, verbose=True)

# Local Test

In [ ]:
# sample random trainset and testset
trainset, testset = train_test_split(data, test_size=.1)

#ls_pu = [0.005,0.01,0.05,0.5]
#ls_qi = [0.01, 0.07, 0.24, 1]
#ls_bu = [0.005, 0.1]
#ls_bi = [0.035, 0.7]
#for pu in ls_pu:
#    for qi in ls_qi:
#        for bu in ls_bu:
#            for bi in ls_bi:
#                print(pu,qi,bu,bi)
#algo1 = SVD(lr_all = 0.002, reg_pu = 0.005, reg_qi = 0.24, reg_bu = 0.005, reg_bi = 0.7)
                # Train the algorithm on the trainset, and predict ratings for the testset
algo4.fit(trainset)
predictions = algo4.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


# Parameter Selection by CV

In [23]:
from surprise import *
from surprise import Dataset
from surprise.model_selection import GridSearchCV

ls_lr = []
ls_reg_pu = []
ls_reg_qi = []

for i in range(10):
    

param_grid = {'lr_all': np.linspace(0.001,0.01,10), 'reg_pu':np.linspace(0.01,0.1,10):np.linspace,'reg_qi':np.linspace(0.01,0.7,10)}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=4,n_jobs = -2)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

SyntaxError: invalid syntax (<ipython-input-23-80575c0772db>, line 5)

SyntaxError: invalid syntax (<ipython-input-25-7c1a4b46bb30>, line 1)

## Create Submission
Note that `ratings` is a sparse matrix that in the shape of (num_items, num_users)

In [20]:
# Prediction
trainset = data.build_full_trainset()

In [21]:
import math

def predict(submission_row_col,algo,trainset):
    algo.fit(trainset)
    predictions = []
    num_neighbours = []
#    details = {}
    for row, col in submission_row_col:
        iid = str(row-1)
        uid = str(col-1)
        details = algo.predict(uid, iid, r_ui=4, verbose=False)
#        num_neighbours.append(details[4]['actual_k'])
        value_pred = float(details[3])
        if value_pred < 1:
            value_pred = 1
        elif value_pred > 5:
            value_pred = 5
        else:
            value_pred = round(value_pred)
        predictions.append(value_pred) 
    return predictions#,num_neighbours

In [22]:
algo1 = SVD(lr_all = 0.002, reg_pu = 0.005, reg_qi = 0.24, reg_bu = 0.005, reg_bi = 0.7)

In [23]:
ratings_pred= predict(submission_row_col, algo1,trainset)
create_csv_submission(submission_pos, ratings_pred, "pred_algo1")

In [56]:
num_neighbours.sort()
num_neighbours.reverse()
#print(num_neighbours[0:10000])

[289, 284, 283, 281, 278, 277, 276, 275, 275, 275, 275, 275, 274, 274, 274, 274, 273, 273, 273, 273, 272, 271, 271, 271, 271, 270, 270, 269, 269, 269, 269, 268, 268, 268, 268, 268, 268, 268, 267, 267, 267, 267, 267, 267, 267, 267, 267, 267, 266, 266, 266, 266, 266, 266, 266, 266, 266, 266, 265, 265, 265, 265, 265, 265, 265, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 263, 263, 263, 263, 263, 263, 263, 263, 263, 262, 262, 262, 262, 262, 262, 262, 262, 261, 261, 261, 261, 261, 261, 261, 261, 260, 260, 260, 260, 260, 260, 260, 260, 260, 259, 259, 259, 259, 259, 259, 259, 258, 258, 258, 258, 258, 258, 258, 258, 258, 258, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254,